In [1]:
import utils

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [2]:
import os

In [17]:
from vgg16 import Vgg16

vgg = Vgg16()

In [18]:
path = os.path.join('data', 'invasive')
train = os.path.join(path, 'train')
valid = os.path.join(path, 'valid')
test = os.path.join(path, 'test')

In [19]:
batch_size = 64

In [20]:
train_batches = vgg.get_batches(train, batch_size=batch_size)

Found 2095 images belonging to 2 classes.


In [21]:
valid_batches = vgg.get_batches(valid, batch_size=batch_size)

Found 200 images belonging to 2 classes.


In [22]:
vgg.finetune(train_batches)

In [57]:
vgg.fit(train_batches, valid_batches, nb_epoch=4)


Epoch 1/4
2095/2095 [==============================] - 91s - loss: 0.3390 - acc: 0.8730 - val_loss: 0.2247 - val_acc: 0.9100
Epoch 2/4
2095/2095 [==============================] - 79s - loss: 0.3057 - acc: 0.8797 - val_loss: 0.3028 - val_acc: 0.8750
Epoch 3/4
2095/2095 [==============================] - 79s - loss: 0.2710 - acc: 0.8883 - val_loss: 0.2322 - val_acc: 0.8950
Epoch 4/4
2095/2095 [==============================] - 75s - loss: 0.3264 - acc: 0.8635 - val_loss: 0.2945 - val_acc: 0.8650


In [58]:
test_batches, predictions = vgg.test(test)

Found 1531 images belonging to 1 classes.


In [59]:
import numpy as np

In [60]:
# check for extreme predictions
np.any(predictions == 1) or np.any(predictions == 0)

True

In [61]:
import pandas as pd
pd.read_csv('data/invasive/sample_submission.csv').head()

,name,invasive
0,1,0.5
1,2,0.5
2,3,0.5
3,4,0.5
4,5,0.5


In [62]:
test_batches.filenames[:5]

['unknown/779.jpg',
 'unknown/1261.jpg',
 'unknown/878.jpg',
 'unknown/552.jpg',
 'unknown/400.jpg']

In [63]:
def filenumber(filepath):
    return filepath.replace('unknown/', '').replace('.jpg', '')

In [64]:
submission_rows = [(filenumber(number), pred[0]) for number, pred in zip(test_batches.filenames, predictions)]

df = pd.DataFrame(submission_rows)
df.columns = ['name', 'invasive']
df.head()

,name,invasive
0,779,0.998836
1,1261,0.073622
2,878,0.023731
3,552,0.998562
4,400,0.617610


In [67]:
df.to_csv('data/invasive/submission.csv', index=False)

In [68]:
#vgg.model.save_weights('models/invasive/vgg16finetuned a8668 v9200 t95726')
#vgg.model.save_weights('models/invasive/vgg16finetuned a8635 v8650 t95565')
